In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\ML_projects\\Wine-Quality-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\ML_projects\\Wine-Quality-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str

In [6]:
from wine_quality_prediction.constants import *
from wine_quality_prediction.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH, 
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path= config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = params,
            target_column = schema.name
        )
        
        return model_evaluation_config

In [12]:
import os
import pandas as pd
import numpy as np
import joblib
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [15]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        return rmse, mae, r2
    
    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_X = test_data.drop([self.config.target_column], axis=1)
        test_Y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_X)
        
        (rmse, mae, r2) = self.eval_metrics(test_Y, predicted_qualities)
        
        # saving metrics as local
        scores = {"rmse : ": rmse, "mae : ": mae, "r2 : ": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [17]:
# pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config= model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e

[2025-10-28 10:27:57,890: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-28 10:27:57,894: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-28 10:27:57,902: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-28 10:27:57,905: INFO: common: created directory at : artifacts]
[2025-10-28 10:27:57,908: INFO: common: created directory at : artifacts/model_evaluation]
[2025-10-28 10:27:57,929: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
